In [743]:
import pandas as pd
# Read the data
df = pd.read_csv("NSDQ100.csv",header = None)
print(df.head())

            0     1      2          3       4           5
0  Activision  atvi  0.56%    Tuesday   4-May  4:05 PM ET
1       Adobe  adbe  0.99%    Tuesday  22-Jun  4:05 PM ET
2    Advanced   amd  3.12%    Tuesday  27-Apr  4:10 PM ET
3     Alexion  alxn  0.47%     Monday   3-May  8:00 AM ET
4       Align  algn  0.12%  Wednesday  28-Apr  4:00 PM ET


In [744]:
# Set the column names
df.columns = ["name", "ticker", "weight","earning day" ,"earning date", "earning time"]
print(df.head())

         name ticker weight earning day earning date earning time
0  Activision   atvi  0.56%     Tuesday        4-May   4:05 PM ET
1       Adobe   adbe  0.99%     Tuesday       22-Jun   4:05 PM ET
2    Advanced    amd  3.12%     Tuesday       27-Apr   4:10 PM ET
3     Alexion   alxn  0.47%      Monday        3-May   8:00 AM ET
4       Align   algn  0.12%   Wednesday       28-Apr   4:00 PM ET


# Task-1


In [745]:
# Exclude all records(companies) whose earning date is before today. (Example: if the earning day is 15 of Jan and today is 20 of Jan, then do not include the record)
import datetime
# Today's Date
today = datetime.datetime.now()
# Convert the earning date string to datetime and filter the recods
df["earning datetime"] = pd.to_datetime(df['earning date'], format="%d-%b").apply(lambda x:x.replace(year=today.year))
excluded_df=df[df["earning datetime"]<today]
result_df = excluded_df.drop("earning datetime",axis =1)
print(result_df.head())

         name ticker weight earning day earning date earning time
0  Activision   atvi  0.56%     Tuesday        4-May   4:05 PM ET
2    Advanced    amd  3.12%     Tuesday       27-Apr   4:10 PM ET
3     Alexion   alxn  0.47%      Monday        3-May   8:00 AM ET
4       Align   algn  0.12%   Wednesday       28-Apr   4:00 PM ET
5    Alphabet   goog  1.11%     Tuesday       27-Apr   4:15 PM ET


In [746]:
# Group all companies by same earning date
# convert weight string to numeric value
import warnings
warnings.filterwarnings("ignore")
excluded_df["weight"] = excluded_df["weight"].str.replace("%","")
excluded_df["weight"] = pd.to_numeric(excluded_df["weight"])
grouped = excluded_df.groupby(by="earning datetime")
# To get accumulative weight, add all values of weights in the group.
accumulated_weights = grouped.agg({"weight":"sum"})
accumulated_weights = accumulated_weights.rename(columns={"weight":"accumulative weight"})
# Group the data in the original dataframe
excluded_df = excluded_df.set_index(["earning datetime","name"])
excluded_df = excluded_df.sort_index()
# Print the accumulated weights for each group
print(accumulated_weights)
# accumulated_weights = accumulated_weights["accumulative weight"]

                  accumulative weight
earning datetime                     
2021-04-21                       5.45
2021-04-22                       6.39
2021-04-26                       6.86
2021-04-27                      15.18
2021-04-28                       9.34
2021-04-29                      21.69
2021-04-30                       0.73
2021-05-03                       0.47
2021-05-04                       6.52
2021-05-05                       5.42
2021-05-06                       6.47
2021-05-10                       1.30
2021-05-11                       0.16
2021-05-17                       6.15
2021-05-18                       1.09
2021-05-19                       3.60
2021-05-20                       2.72
2021-05-25                       0.53
2021-05-26                       4.31
2021-05-27                       4.37
2021-05-31                       2.58
2021-06-01                       0.37
2021-06-02                       2.97
2021-06-03                       1.78


In [747]:
# Keep groups, whose accumulative weight is above or equal to 10%.
filtered_df = excluded_df.copy()
# Join the original data with groups with their accumulated weights
dates = excluded_df.index.get_level_values('earning datetime')
filtered_df['accumulative weight'] = accumulated_weights.loc[dates].values
# Filter by weights
filtered_df = filtered_df[filtered_df["accumulative weight"]>10]
filtered_df["accumulative weight"] = filtered_df["accumulative weight"].apply(lambda x:str(x)+"%")
filtered_df = filtered_df.reset_index()
filtered_df = filtered_df.set_index(["earning datetime","accumulative weight","name"])
print(filtered_df)

                                                ticker  weight earning day  \
earning datetime accumulative weight name                                    
2021-04-27       15.18%              Advanced      amd    3.12     Tuesday   
                                     Alphabet     goog    1.11     Tuesday   
                                     Alphabet    googl    1.13     Tuesday   
                                     Amgen        amgn    1.44     Tuesday   
                                     Fiserv       fisv    0.15     Tuesday   
                                     Illumina     ilmn    1.26     Tuesday   
                                     Maxim        mxim    1.77     Tuesday   
                                     Microsoft    msft    1.31     Tuesday   
                                     Mondelez     mdlz    0.03     Tuesday   
                                     PACCAR       pcar    1.52     Tuesday   
                                     Starbucks    sbux    0.70  

In [748]:
# Order the groups by earning dates and print the result
filtered_df = filtered_df.sort_index(level=1)
result = filtered_df.droplevel("name").index.to_frame(index=False)
result = result.drop_duplicates("earning datetime").reset_index().drop("index",axis=1)
print(result)

  earning datetime accumulative weight
0       2021-04-27              15.18%
1       2021-04-29              21.69%


# Task-2


In [749]:
# a) Rewrite the tasks from threshold part. And this time, include all the data about the groups: name, ticker, weight, earning data, earning time.
task_data = filtered_df.copy()
print(task_data)

                                                ticker  weight earning day  \
earning datetime accumulative weight name                                    
2021-04-27       15.18%              Advanced      amd    3.12     Tuesday   
                                     Alphabet     goog    1.11     Tuesday   
                                     Alphabet    googl    1.13     Tuesday   
                                     Amgen        amgn    1.44     Tuesday   
                                     Fiserv       fisv    0.15     Tuesday   
                                     Illumina     ilmn    1.26     Tuesday   
                                     Maxim        mxim    1.77     Tuesday   
                                     Microsoft    msft    1.31     Tuesday   
                                     Mondelez     mdlz    0.03     Tuesday   
                                     PACCAR       pcar    1.52     Tuesday   
                                     Starbucks    sbux    0.70  

In [750]:
from pytz import timezone
gmt = pytz.timezone('GMT')
et = pytz.timezone('US/Eastern')
task_data["earning time"] = task_data['earning time'].str.replace("0:00 AM ET","12:00 PM ET")
task_data = task_data[(task_data["earning time"] != "After the close")]
task_data["earning_time(gmt)"] = pd.to_datetime(task_data['earning time'], format="%I:%M %p ET").apply(lambda x:x.replace(year=today.year))
task_data= task_data[~task_data["earning_time(gmt)"].isnull()]
task_data["earning_time(gmt)"] = task_data["earning_time(gmt)"].apply(lambda x:x.replace(tzinfo=et).astimezone(gmt))
task_data["earning time"] = task_data["earning_time(gmt)"].apply(lambda x:x.strftime("%I:%M %p GMT"))
task_data = task_data.drop("earning_time(gmt)",axis=1) 
print(task_data)

                                                ticker  weight earning day  \
earning datetime accumulative weight name                                    
2021-04-27       15.18%              Advanced      amd    3.12     Tuesday   
                                     Alphabet     goog    1.11     Tuesday   
                                     Alphabet    googl    1.13     Tuesday   
                                     Amgen        amgn    1.44     Tuesday   
                                     Fiserv       fisv    0.15     Tuesday   
                                     Illumina     ilmn    1.26     Tuesday   
                                     Maxim        mxim    1.77     Tuesday   
                                     Microsoft    msft    1.31     Tuesday   
                                     Mondelez     mdlz    0.03     Tuesday   
                                     PACCAR       pcar    1.52     Tuesday   
                                     Starbucks    sbux    0.70  

In [751]:
task_data["AM/PM"] = task_data["earning time"].apply(lambda x:"AM" if "AM" in x else "PM")
task_data = task_data.reset_index()
task_data = task_data.set_index(["earning datetime","accumulative weight","AM/PM","name"]).sort_index()
print(task_data)
# To convert to csv use 
task_data.to_csv("task_2.csv")

                                                      ticker  weight  \
earning datetime accumulative weight AM/PM name                        
2021-04-27       15.18%              PM    Advanced      amd    3.12   
                                           Alphabet     goog    1.11   
                                           Alphabet    googl    1.13   
                                           Amgen        amgn    1.44   
                                           Fiserv       fisv    0.15   
                                           Illumina     ilmn    1.26   
                                           Maxim        mxim    1.77   
                                           Microsoft    msft    1.31   
                                           Mondelez     mdlz    0.03   
                                           PACCAR       pcar    1.52   
                                           Starbucks    sbux    0.70   
                                           Texas         txn    